In [1]:
import os
import sys

import joblib
import numpy as np
import pandas as pd
import lightgbm as lgb
import tensorflow as tf

from interpret.glassbox import ExplainableBoostingRegressor
from mapie.regression import MapieQuantileRegressor
from quantile_forest import RandomForestQuantileRegressor

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..')))

from src_new.utils.model import split_dataset, compare_models_per_station, create_deep_model

2025-04-26 16:29:09.385571: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-26 16:29:09.527249: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-26 16:29:09.581449: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-26 16:29:09.597298: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-26 16:29:09.697510: I tensorflow/core/platform/cpu_feature_guar

##### Constants :
- **INPUT_DIR**: Directory for input data (same as in "02 - Feature Engineering").
- **MODEL_DIR**: Directory where trained models are saved.
- **DATASET_DIR**: Directory where the Zenodo dataset is unzipped.

##### Model Parameters

- **SEED**: 42 (for reproducibility)
- **NUMBER_OF_WEEK**: 4 (one model is trained per week)

##### FINAL_MODELS

- **mapie**: Combines LightGBM with MAPIE. **MAPIE** (Model Agnostic Prediction Interval Estimator) computes prediction intervals for any regression model using conformal methods.
- **qrf**: Quantile Random Forest (natively produces prediction intervals)
- **ebm**: Explainable Boosting Machine is used as a exemple that does not natively implement prediction intervals, but that can be customised to do so.

In [2]:
INPUT_DIR = "../../../data/input/"
MODEL_DIR = "../../../models/"
DATASET_DIR = "../../../dataset/"

SEED = 40
NUMBER_OF_WEEK = 4 # Number of weeks to predict one model is trained per week

FINAL_MODELS = ["qrf",
                # "mapie",
                #"ebm",
                #"deep_ensemble",
                ]
mapie_enbpi = {}
mapie = {}
qrf = {}
mapie_aci = {}

COLUMNS_TO_DROP = ["water_flow_week1", "water_flow_week2", "water_flow_week3", "water_flow_week4"]

dataset_train = pd.read_csv(f"{INPUT_DIR}dataset_baseline.csv")

dataset_train = dataset_train.set_index("ObsDate")

if not os.path.exists(f"{MODEL_DIR}final/"):
    os.makedirs(f"{MODEL_DIR}final/")

dataset_train_2 = dataset_train[dataset_train.station_code.isin([56659998,56994500]) ]#   & (dataset_train.index>="1997-12")
X_train_2 = dataset_train_2.drop(columns=COLUMNS_TO_DROP)
y_train_2 = {}
for i in range(0, NUMBER_OF_WEEK):
    y_train_2[i] = dataset_train_2[f"water_flow_week{i+1}"]

dataset_train = dataset_train[(~dataset_train.station_code.isin([56659998,56994500]))  & (dataset_train.index<="1998-12-31")
] #
X_train = dataset_train.drop(columns=COLUMNS_TO_DROP)
y_train = {}
for i in range(0, NUMBER_OF_WEEK):
    y_train[i] = dataset_train[f"water_flow_week{i+1}"]


### 2. Data Loading
Load in the baseline datasets, create the directory to save models.

In [3]:
ALPHA = 0.1

In [4]:
X_train_qrf = X_train.drop(columns=["station_code"])

if "qrf" in FINAL_MODELS:
    for i in range(NUMBER_OF_WEEK):
        print(f"Training week {i}")
        # Train RandomForestQuantileRegressor
        qrf[i] = RandomForestQuantileRegressor(n_estimators=50, max_depth=7, min_samples_leaf=6, n_jobs=-1, random_state=SEED)
        qrf[i].fit(X_train_qrf, y_train[i])

        #time = pd.Timestamp.now().strftime("%Y-%m-%d_%H-%M-%S")
        #model_path = f"{MODEL_DIR}final/qrf_quantile_{time}_week_{i}.pkl"
        #joblib.dump(qrf[i], model_path)

Training week 0
Training week 1
Training week 2
Training week 3


In [5]:
conformity_score = []
for i in range(4):
    y_pis_qrf = qrf[i].predict(X_train_2.drop(columns=["station_code"]), quantiles=[ALPHA/2, 1-ALPHA/2])
    conformity_score.append(np.quantile(np.maximum(y_pis_qrf[:, 0] - y_train_2[i], y_train_2[i] - y_pis_qrf[:, 1]), 1 - ALPHA))

print("maximum non conformity score is: {:.1f}".format(max(conformity_score)))

/home/jfan/env/venv/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestQuantileRegressor was fitted without feature names
  warnings.warn(
/home/jfan/env/venv/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestQuantileRegressor was fitted without feature names
  warnings.warn(
/home/jfan/env/venv/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestQuantileRegressor was fitted without feature names
  warnings.warn(
/home/jfan/env/venv/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestQuantileRegressor was fitted without feature names
  warnings.warn(


maximum non conformity score is: 9.7
